# Intuit Quickbooks Upgrade

* Team-lead GitLab userid:
* Group name:
* Team member names:

## Setup

Please complete this python notebook with your group by answering the questions in `intuit.pdf`. Create a Notebook and HTML file with all your results and comments and push both the Notebook and HTML file to GitLab when your team is done. All results MUST be reproducible (i.e., the TA and I must be able to recreate the HTML file from the Jupyter Notebook without changes or errors). This means that you should NOT use any python-packages that are not part of the rsm-msba-spark docker container.

This is the first group assignment for MGTA 455 and you will be using Git and GitLab. If two people edit the same file at the same time you could get what is called a "merge conflict". This is not something serious but you should realize that Git will not decide for you who's change to accept so the team-lead will have to determine the edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and then push them to GitLab. Make "pull first" a habit!

If multiple people are going to work on the assignment at the same time I recommend you work in different notebooks. You can then `%run ...`  these "sub" notebooks from the main assignment file. You can seen an example of this in action below for the `question1.ipynb` notebook

Some group work-flow tips:

* Pull, edit, save, stage, commit, and push
* Schedule who does what and when
* Try to avoid working simultaneously on the same file 
* If you are going to work simultaneously, do it in different notebooks, e.g., 
    - question1.ipynb, question2.ipynb, question3.ipynb
* Use the `%run ... ` command to bring different pieces of code together into the main jupyter notebook
* Put python functions in modules that you can import from your notebooks. See the example below for the `example` function defined in `utils/functions.py`

A graphical depiction of the group work-flow is shown below:

![](images/git-group-workflow-wbg.png)

Tutorial videos about using Git, GitLab, and GitGadget for group assignments:

* Setup the MSBA server to use Git and GitLab: https://youtu.be/zJHwodmjatY
* Dealing with Merge Conflicts: https://youtu.be/qFnyb8_rgTI
* Group assignment practice: https://youtu.be/4Ty_94gIWeA

In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

# increase plot resolution
# mpl.rcParams["figure.dpi"] = 150

In [10]:
## loading the data - this dataset must NOT be changed
intuit75k = pd.read_pickle("data/intuit75k.pkl")
intuit75k["res1_yes"] = (intuit75k["res1"] == "Yes").astype(int)
intuit75k.head()

id    zip  zip_bins      sex  bizflag  numords  dollars  last  sincepurch  \
0   1  94553        18     Male        0        2    109.5     5          12   
1   2  53190        10  Unknown        0        1     69.5     4           3   
2   3  37091         8     Male        0        4     93.0    14          29   
3   4  02125         1     Male        0        1     22.0    17           1   
4   5  60201        11     Male        0        1     24.5     2           3   

   version1  owntaxprod  upgraded res1  training  res1_yes  
0         0           0         0   No         1         0  
1         0           0         0   No         0         0  
2         0           0         1   No         0         0  
3         0           0         0   No         1         0  
4         0           0         0   No         0         0

In [11]:
# show dataset description
rsm.describe(intuit75k)

## Intuit: Quickbooks upgrade

The purpose of this exercise is to gain experience modeling the response to an upsell campaign. The `intuit75k.rds` file contains data on 75,000 (small) businesses that were selected randomly from the 801,821 that were sent the wave-1 mailing. The mailing contained an offer to upgrade to the latest version of the Quickbooks software. 

Variable `res1` denotes which of these businesses responded to the mailing by purchasing Quickbooks version 3.0 from Intuit Direct. Note that Intuit Direct sells products directly to its customers rather than through a retailer. Use the available data to predict which businesses that did not respond to the wave-1 mailing, are most likely to respond to the wave-2 mailing. Note that variables were added, deleted, and recoded so please ignore the variable descriptions in Exhibit 3 in the case in the course reader. Instead, use the variable descriptions below:

## Variable description

* id: Small business customer ID
* zip: 5-

In [16]:
intuit75k.describe()

id      zip_bins       bizflag       numords       dollars  \
count  75000.000000  75000.000000  75000.000000  75000.000000  75000.000000   
mean   37500.500000     10.498333      0.245080      2.076280     93.087133   
std    21650.779432      5.765751      0.430137      1.241355     81.205900   
min        1.000000      1.000000      0.000000      1.000000      1.000000   
25%    18750.750000      5.000000      0.000000      1.000000     37.000000   
50%    37500.500000     10.000000      0.000000      2.000000     71.000000   
75%    56250.250000     15.000000      0.000000      3.000000    124.500000   
max    75000.000000     20.000000      1.000000      5.000000   1149.000000   

               last    sincepurch     version1    owntaxprod      upgraded  \
count  75000.000000  75000.000000  75000.00000  75000.000000  75000.000000   
mean      15.838427     15.657173      0.21400      0.028693      0.208387   
std        9.539057     10.026312      0.41013      0.166944      0.406157   
min        1.000000      1.000000      0.00000      0.000000      0.000000   
25%        8.000000      7.000000      0.00000      0.000000      0.000000   
50%       15.000000     14.000000      0.00000      0.000000      0.000000   
75%       24.000000     24.000000      0.00000      0.000000      0.000000   
max       36.000000     36.000000      1.00000      1.000000      1.000000   

           training      res1_yes  
count  75000.000000  75000.000000  
mean       0.700000      0.048013  
std        0.458261      0.213796  
min        0.000000      0.000000  
25%        0.000000      0.000000  
50%        1.000000      0.000000  
75%        1.000000      0.000000  
max        1.000000      1.000000

In [19]:
intuit75k['sex'].value_counts()

Male       49507
Female     15763
Unknown     9730
Name: sex, dtype: int64

In [21]:
intuit75k['res1'].value_counts()

No     71399
Yes     3601
Name: res1, dtype: int64

In [5]:
# run python code from another notebook
%run ./sub-notebooks/question1.ipynb

Exception: File `'./sub-notebooks/question1.ipynb.py'` not found.

In [ ]:
# importing functions from a module/package
from utils import functions
functions.example()

In [6]:
## Question answers


In [ ]:
1. Exploration
2. Standardize
3. Modelling
4. Interpretation
5. Performance function - profit, AUC, Accuracy/precision/recall 
6. VIF/correlation/
7. Lasso